In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time
import csv
import numpy as np
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/title/tt23778430/reviews/?ref_=tt_urv_sm")
time.sleep(30)

# Nhấn nút "Load More" nếu có
while True:
    try:
        load_more_button = driver.find_element(By.CLASS_NAME, 'ipl-load-more__button')
        load_more_button.click()
        time.sleep(5)  # Chờ để nội dung tải
    except:
        break  # Không còn nút "Load More", thoát khỏi vòng lặp

In [8]:
ratings = driver.find_elements(By.CLASS_NAME, 'ipl-ratings-bar')
ratings_list = []

# Trích xuất điểm số từ từng rating
for rating in ratings:
    try:
        score = rating.text.split("/")[0]  # Lấy điểm trước dấu "/"
        ratings_list.append(float(score))
    except:
        continue

# Lấy danh sách ngày review
review_date = driver.find_elements(By.CLASS_NAME, 'review-date')
review_date_list = [i.text for i in review_date]

# Lấy danh sách nội dung review và xử lý
a = driver.find_elements(By.CLASS_NAME, "content")
reviews = [
    re.sub(
        r"\n\d+ out of \d+ found this helpful\. Was this review helpful\? Sign in to vote\.\nPermalink", 
        "", 
        i.text
    ).strip() 
    for i in a
]

In [9]:

# Ghi danh sách vào file CSV
with open(r"G:\test\data_nckh\data\worse\comedy\the_fabulous_four.csv", mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    
    # Ghi tiêu đề cột
    writer.writerow(["Rating", "Review_Date", "Review"])
    
    # Ghi từng review vào tệp CSV
    for i in range(0, len(reviews)):
        if i < len(ratings_list):
            # Nếu có đủ rating, ghi vào file
            writer.writerow([ratings_list[i], review_date_list[i], reviews[i]])
        else:
            # Nếu thiếu rating, dùng trung bình của các giá trị hiện có
            writer.writerow([round(np.mean(ratings_list), 2), review_date_list[i], reviews[i]])


In [10]:
driver.quit()
